# `Mini Project II`
## Points of Interest in Playa Del Carmen, Mexico

### Ali Bahrami
#### November 04, 2021

In [2]:
import requests as re
import pandas as pd
import sqlite3 as sql
import json
import os
from IPython.display import JSON

In [3]:
## Pull the Data


In [4]:
## Foursquare

## Run only once so as not to overload the API. Uncomment resp variable below to run again

client_id = os.environ['client_id']
client_secret = os.environ["client_secret"]

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=client_id,
client_secret=client_secret,
near='Playa Del Carmen, Mexico',
section='topPicks',
limit=50,
sortByPopularity = 1,
v=20211104)

resp = re.get(url=url, params=params)
data = json.loads(resp.text)

#Write to file
with open('#########################################', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# Shows same data in a friendlier format only in Jupyter Notebook    
JSON(data)

<IPython.core.display.JSON object>

In [5]:
with open('data_fs.json','r') as f:
    data = json.loads(f.read())

dfs = pd.json_normalize(data, record_path=['response', 'groups', 'items'], errors='ignore')

dfs = dfs.applymap(str)
dfs.dtypes


FileNotFoundError: [Errno 2] No such file or directory: 'data_fs.json'

In [31]:
dfs.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.labeledLatLngs', 'venue.venuePage.id',
       'flags.outsideRadius', 'venue.location.neighborhood'],
      dtype='object')

In [4]:
conn = sql.connect('playa.db')
dfs.to_sql('foursquare_data2', conn, schema='online', index=False, if_exists='append')

In [73]:
dfs_sql = pd.read_sql('SELECT * FROM foursquare_data2', conn)
dfs_sql[['venue.name', 'reasons.items', 'venue.location.formattedAddress']].iloc[11:20]

,venue.name,reasons.items,venue.location.formattedAddress
11,La Bodeguita del Medio,"[{'summary': 'This spot is popular', 'type': '...","['5ta Avenida (Esq. con 34 Nte.)', '77710 Play..."
12,La Cochi-loka,"[{'summary': 'This spot is popular', 'type': '...","['Calle 10 (Avenida 10)', 'Playa Del Carmen, Q..."
13,Dac,"[{'summary': 'This spot is popular', 'type': '...","['30 Av. Norte LB', 'Playa del Carmen, Quintan..."
14,Taqueria El Ñero,"[{'summary': 'This spot is popular', 'type': '...","['Av', 'Playa del Carmen, Quintana Roo', 'Méxi..."
15,Chiltepin Marisquillos,"[{'summary': 'This spot is popular', 'type': '...","['20 nte (Av 34 Nte)', '77720 Playa del Carmen..."
16,Las Helodias,"[{'summary': 'This spot is popular', 'type': '...","['Calle 28 Nte (1 Av. Nte)', '77710 Playa del ..."
17,Caiman Tugurio,"[{'summary': 'This spot is popular', 'type': '...","['24 Avenue (5ta)', 'Playa del Carmen, QROO', ..."
18,El Fogón Fiesta Jarocha,"[{'summary': 'This spot is popular', 'type': '...","['Av. 30 (Calle 6 Bis)', '77710 Playa Del Carm..."
19,Renzo's Pizza,"[{'summary': 'This spot is popular', 'type': '...","['Quintana Roo', 'México']"


In [49]:
# Yelp
## Run only once so as not to overload the API. Uncomment resp variable below to run again

api_key = os.environ['yelp_key']

url = 'https://api.yelp.com/v3/businesses/search'

headers= {"Authorization": f"Bearer {api_key}"}

params = dict(
location = 'Playa Del Carmen, Mexico',
limit=50,
sort_by = 'review_count'
)
resp = re.get(url=url, headers=headers, params=params)
data_yelp = json.loads(resp.text)

#Write to file
with open('####################################', 'w', encoding='utf-8') as f:
    json.dump(data_yelp, f, ensure_ascii=False, indent=4)
    
JSON(data_yelp)

<IPython.core.display.JSON object>

In [6]:
with open('data_yelp.json','r') as f:
    data_yelp = json.loads(f.read())
df_yelp = pd.json_normalize(data_yelp, record_path=['businesses'], errors='ignore')
df_yelp.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [7]:

df_yelp = df_yelp.applymap(str)
df_yelp.dtypes

id                          object
alias                       object
name                        object
image_url                   object
is_closed                   object
url                         object
review_count                object
categories                  object
rating                      object
transactions                object
price                       object
phone                       object
display_phone               object
distance                    object
coordinates.latitude        object
coordinates.longitude       object
location.address1           object
location.address2           object
location.address3           object
location.city               object
location.zip_code           object
location.country            object
location.state              object
location.display_address    object
dtype: object

In [70]:
conn = sql.connect('playa.db')
df_yelp.to_sql('yelp_data', conn, schema='online', index=False, if_exists='append')

In [74]:
dyelp_sql = pd.read_sql('SELECT * FROM yelp_data', conn)
dyelp_sql[['name', 'rating', 'review_count', 'price','is_closed',  'categories','location.display_address']].head(10)

,name,rating,review_count,price,is_closed,categories,location.display_address
0,El Fogón,4.5,928,$,False,"[{'alias': 'tacos', 'title': 'Tacos'}]","['30 Avenida Nte. S/N', 'Col. Centro', '77710 ..."
1,Kaxapa Factory,5.0,545,$,False,"[{'alias': 'latin', 'title': 'Latin American'}...","['Calle 10 Norte S/N', 'Col. Centro', '77710 P..."
2,Madrez Café,5.0,463,$,False,"[{'alias': 'breakfast_brunch', 'title': 'Break...","['Calle 8 Nte S/N', '77710 Playa del Carmen, Q..."
3,Xcaret Park,4.0,420,nan,False,"[{'alias': 'amusementparks', 'title': 'Amuseme...","['Carretera Chetumal-Puerto Juárez Km 282', '7..."
4,La Cueva del Chango,4.5,340,$$,False,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...","['Calle 38 Nte. S/N', 'Col. Zazil Ha', '77710 ..."
5,Xplor Park,4.5,310,nan,False,"[{'alias': 'amusementparks', 'title': 'Amuseme...","['Carretera Federal 307 Km 282', '77710 Playa ..."
6,Alux Restaurant,3.5,269,$$$$,False,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","['Av. Juárez S/N', 'Col. Ejidal', '77710 Playa..."
7,La Choza,4.5,236,$$,False,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","['10a Avenida Sur 216', 'Col. Centro', '77600 ..."
8,Los Aguachiles,4.5,235,$$,False,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","['Calle 34 Nte S/N', 'Col. Centro', '77710 Pla..."
9,La Famiglia,4.5,226,$$,False,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","['Av. 10 S/N', 'Col. Centro', '77710 Playa del..."


In [27]:
api_key = os.environ['maps_api_key']

In [6]:
# Google Places API
api_key = os.environ['maps_api_key']

url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'


params = dict(
query = 'Attractions in Playa Del Carmen, Mexico',
key=api_key
)
resp = re.get(url=url, params=params)

data_google = json.loads(resp.text)




In [35]:
#Write to file
with open('data_google.json', 'w', encoding='utf-8') as f:
    json.dump(data_google, f, ensure_ascii=False, indent=4)
    
JSON(data_google)

<IPython.core.display.JSON object>

In [37]:
with open('data_google.json','r') as f:
    data_google = json.loads(f.read())
df_google = pd.json_normalize(data_google, record_path=['results'], errors='ignore')
df_google.columns

Index(['business_status', 'formatted_address', 'icon', 'icon_background_color',
       'icon_mask_base_uri', 'name', 'photos', 'place_id', 'rating',
       'reference', 'types', 'user_ratings_total', 'geometry.location.lat',
       'geometry.location.lng', 'geometry.viewport.northeast.lat',
       'geometry.viewport.northeast.lng', 'geometry.viewport.southwest.lat',
       'geometry.viewport.southwest.lng', 'plus_code.compound_code',
       'plus_code.global_code', 'opening_hours.open_now',
       'permanently_closed'],
      dtype='object')

In [38]:
df_google = df_google.applymap(str)
df_google.dtypes

business_status                    object
formatted_address                  object
icon                               object
icon_background_color              object
icon_mask_base_uri                 object
name                               object
photos                             object
place_id                           object
rating                             object
reference                          object
types                              object
user_ratings_total                 object
geometry.location.lat              object
geometry.location.lng              object
geometry.viewport.northeast.lat    object
geometry.viewport.northeast.lng    object
geometry.viewport.southwest.lat    object
geometry.viewport.southwest.lng    object
plus_code.compound_code            object
plus_code.global_code              object
opening_hours.open_now             object
permanently_closed                 object
dtype: object

In [39]:
conn = sql.connect('playa.db')
df_google.to_sql('google_data', conn, schema='online', index=False, if_exists='append')

In [60]:
dgoogle_sql = pd.read_sql('SELECT * FROM google_data', conn)
dgoogle_sql[['name', 'rating', 'user_ratings_total', 'business_status', ['types'][0],'formatted_address']].tail(10)

,name,rating,user_ratings_total,business_status,types,formatted_address
10,KaribbeanMermaids,5.0,6,OPERATIONAL,"['tourist_attraction', 'point_of_interest', 'e...","Carretera Federal, Calle 22 Nte, entre calles ..."
11,Cenote Chaak Tun,4.7,1519,OPERATIONAL,"['tourist_attraction', 'park', 'point_of_inter...","Av. Juárez prolongación km 4.7, Ejidal, 77723 ..."
12,3D Museum of Wonders,4.5,1375,OPERATIONAL,"['museum', 'tourist_attraction', 'point_of_int...","10 Avenida Nte., Centro, 77710 Playa del Carme..."
13,Mexxtremo Adventures,4.5,140,OPERATIONAL,"['tourist_attraction', 'travel_agency', 'point...","Calle 90 Nte., Ejidal, 77712 Playa del Carmen,..."
14,Xaman - Ha Aviary,3.9,350,OPERATIONAL,"['zoo', 'tourist_attraction', 'park', 'point_o...","P.º Xaman - Ha, Galaxia del Carmen II, 77717 P..."
15,Xenses,4.8,10491,OPERATIONAL,"['amusement_park', 'tourist_attraction', 'poin...",Carretera Chetumal - Puerto Juárez km 282 MZA ...
16,Río Secreto,4.7,4343,OPERATIONAL,"['tourist_attraction', 'travel_agency', 'point...",Carretera Federal Libre Chetumal- Puerto Juáre...
17,Parque La Ceiba,4.6,1035,CLOSED_TEMPORARILY,"['park', 'movie_theater', 'tourist_attraction'...","Diagonal 60 X calle 1 y, C. 3 Sur s/n, Ejidal,..."
18,MONSTERLAND,5.0,1,OPERATIONAL,"['tourist_attraction', 'point_of_interest', 'e...","Calle Corazón, 5ta Avenida entre 12 Y 14, 5 Av..."
19,"Parque, Portal Maya",4.4,2839,OPERATIONAL,"['park', 'tourist_attraction', 'point_of_inter...","15 Avenida Nte, Centro, 77710 Playa del Carmen..."
